In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import sys
import yaml

sys.path.append("../")

%load_ext autoreload
%autoreload 2

In [2]:
from preprocessing.opp_preprocess import *
from model.hierarchical_self_attention_model import HSA_model_session_guided_window

In [3]:
# def create_windowed_dataset_opp_mid(df, features, class_label, MID_LABEL_COL, LOCO_LABEL_COL, window_size=24, stride = 12):
#     X = df[features].values
#     y = df[class_label].values
#     segments = []
#     labels = []
#     seg_start= 0
#     seg_end = window_size
#     mid_labels = []
#     loco_labels = []
#     while seg_end <= len(X):
#         if len(np.unique(y[seg_start:seg_end])) == 1:
#             segments.append(X[seg_start:seg_end])
#             labels.append(y[seg_start]) 
#             mid_labels.append(df[str(MID_LABEL_COL)].values[seg_start:seg_end])
#             loco_labels.append(df[str(LOCO_LABEL_COL)].values[seg_start:seg_end])

#             seg_start += stride
#             seg_end = seg_start + window_size

#         else:
#             current_label = y[seg_start]
#             for i in range(seg_start, seg_end):
#                 if y[i] != current_label:
#                     seg_start = i
#                     seg_end = seg_start + window_size
#                     break

#     return np.asarray(segments).astype(np.float32), np.asarray(labels), mid_labels, loco_labels

In [4]:
data_path = '/home/hariub/data/HAR/processed/clean_opp_nodrill.csv'
df = pd.read_csv(data_path)

metadata_file = open('../configs/metadata.yaml', mode='r')
metadata = yaml.load(metadata_file, Loader=yaml.FullLoader)['opp_preprocess']
FEATURES = [str(i) for i in range(77)]
LOCO_LABEL_COL = 77
MID_LABEL_COL = 78
HI_LABEL_COL = 79
SUBJECT_ID = 80
RUN_ID = 81

# df = df[df[str(HI_LABEL_COL)] != 0]
df[FEATURES] = df[FEATURES].interpolate(method='linear', axis=0)
df = df.fillna(0)

scaler = StandardScaler()
df[FEATURES] = scaler.fit_transform(df[FEATURES])

BENCHMARK_TEST = ((df[str(SUBJECT_ID)] == 2) | (df[str(SUBJECT_ID)] == 3)) & (
            (df[str(RUN_ID)] == 4) | (df[str(RUN_ID)] == 5))

train_df = df[~ BENCHMARK_TEST]
test_df = df[BENCHMARK_TEST]

SLIDING_WINDOW_LENGTH = 60
SLIDING_WINDOW_STEP = 30
N_WINDOW, N_TIMESTEP = 10, 6

X_train, y_train, m_labels_tr, loco_labels_tr = create_windowed_dataset_opp(train_df, FEATURES, str(
    MID_LABEL_COL), MID_LABEL_COL, LOCO_LABEL_COL, window_size=SLIDING_WINDOW_LENGTH, stride=SLIDING_WINDOW_STEP)
X_test, y_test, m_labels_ts, loco_labels_ts = create_windowed_dataset_opp(test_df, FEATURES, str(
    MID_LABEL_COL), MID_LABEL_COL, LOCO_LABEL_COL, window_size=SLIDING_WINDOW_LENGTH, stride=SLIDING_WINDOW_STEP)

X_train = X_train.reshape(
    (X_train.shape[0], N_WINDOW, N_TIMESTEP, len(FEATURES)))
X_test = X_test.reshape(
    (X_test.shape[0], N_WINDOW, N_TIMESTEP, len(FEATURES)))

y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

In [5]:
np.asarray(m_labels_tr).shape

(14851, 60)

In [6]:
y_train.shape

(14851, 18)

In [7]:
print(X_train.shape)
print(y_train.shape)

(14851, 10, 6, 77)
(14851, 18)


In [8]:
# m_labels_tr = np.asarray(m_labels_tr).reshape((X_train.shape[0], N_WINDOW, N_TIMESTEP))
# m_labels_ts = np.asarray(m_labels_ts).reshape((X_test.shape[0], N_WINDOW, N_TIMESTEP))

In [9]:
# y_train_mid = np.zeros((X_train.shape[0], N_WINDOW))
# for i in range(len(y_train_mid)):
#     for j in range(N_WINDOW):
#         y_train_mid[i,j] = np.bincount(m_labels_tr[i, j]).argmax()
        
# y_test_mid = np.zeros((X_test.shape[0], N_WINDOW))
# for i in range(len(y_test_mid)):
#     for j in range(N_WINDOW):
#         y_test_mid[i,j] = np.bincount(m_labels_ts[i, j]).argmax()

In [10]:
# y_train_mid = tf.keras.utils.to_categorical(y_train_mid)
# y_test_mid = tf.keras.utils.to_categorical(y_test_mid)

In [11]:
y_train_mid = np.repeat(np.expand_dims(y_train, axis=1), repeats=N_WINDOW, axis=1)
# y_val_mid = np.repeat(np.expand_dims(y_val, axis=1), repeats=N_WINDOW, axis=1)
y_test_mid = np.repeat(np.expand_dims(y_test, axis=1), repeats=N_WINDOW, axis=1)

In [12]:
# hparam_file = open('../configs/hyperparameters.yaml', mode='r')
# hyperparameters = yaml.load(hparam_file, Loader=yaml.FullLoader)
# hparams = hyperparameters['HSA_model']
# hparams['modality_indices'] = hparams['modality_indices']['opp']
# hparams['n_window'], hparams['n_timesteps'], hparams['n_features'], hparams['n_outputs'] = X_train.shape[1], X_train.shape[2], X_train.shape[3], y_train.shape[1]

In [13]:
hparam_file = open('../configs/hyperparameters.yaml', mode='r')
hyperparameters = yaml.load(hparam_file, Loader=yaml.FullLoader)
DATASET_NAME = 'opp'
hparams_all = hyperparameters['HSA_model']
hparams = hparams_all[DATASET_NAME]

hparams['n_window'], hparams['n_timesteps'], hparams['n_features'], hparams['n_outputs'] = X_train.shape[1], X_train.shape[2], X_train.shape[3], y_train.shape[1]
hparams['n_outputs_window']=  y_train.shape[1]

## Training

In [14]:
tf.keras.backend.clear_session()
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)
    
device_list = ['/gpu:'+str(i) for i in range(4, 6)]
strategy = tf.distribute.MirroredStrategy(devices=device_list)
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))
with strategy.scope():
    model = HSA_model_session_guided_window(**hparams).get_compiled_model()

6 Physical GPUs, 6 Logical GPUs
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:4', '/job:localhost/replica:0/task:0/device:GPU:5')
Number of devices: 2


In [15]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 10, 6, 77)]  0                                            
__________________________________________________________________________________________________
multi_window_encoder (MultiWind ((None, None, 64), ( 754112      input_1[0][0]                    
__________________________________________________________________________________________________
modality_encoder_block_7 (Modal (None, None, 64)     104000      multi_window_encoder[0][0]       
__________________________________________________________________________________________________
combined_sensor_self_attention_ ((None, 64), (None,  49856       modality_encoder_block_7[0][0]   
____________________________________________________________________________________________

In [16]:
model.fit(X_train, [y_train_mid, y_train], batch_size=len(device_list) * 64, epochs=30, validation_split=0.1, use_multiprocessing=True, callbacks=[model_checkpoint_callback])

Epoch 1/30
INFO:tensorflow:batch_all_reduce: 288 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/devi

InternalError: 2 root error(s) found.
  (0) Internal:  NCCL: unhandled system error. Set NCCL_DEBUG=WARN for detail.
	 [[node NcclAllReduce (defined at <ipython-input-16-0c1953b967a9>:1) ]]
	 [[GroupCrossDeviceControlEdges_0/Identity_5/_1669]]
  (1) Internal:  NCCL: unhandled system error. Set NCCL_DEBUG=WARN for detail.
	 [[node NcclAllReduce (defined at <ipython-input-16-0c1953b967a9>:1) ]]
0 successful operations.
1 derived errors ignored. [Op:__inference_train_function_78128]

Function call stack:
train_function -> train_function


In [ ]:
pred_mid, pred_sess = model.predict(X_test, batch_size=len(device_list) * 64)

In [ ]:
print(classification_report(np.argmax(y_test, axis=1), np.argmax(pred_sess, axis=1)))

In [ ]:
import json
activity_map = json.load(open(os.path.join('..','data', 'activity_maps', 'opp_activity.json')))

In [ ]:
activity_map

In [ ]:
confm = confusion_matrix(np.argmax(y_test, axis=1), np.argmax(pred_sess, axis=1))
activity_list = list(activity_map.values())
df_cm = pd.DataFrame(confm, index=activity_list, columns=activity_list)
plt.figure(figsize = (10,8))
sns.heatmap(df_cm, annot=True, fmt='d', cmap="YlGnBu")

In [ ]:
print(classification_report(np.argmax(y_test_mid.reshape(-1, 18), axis=1), np.argmax(pred_mid.reshape(-1, 18), axis=1), target_names=mid_activity_opp))

In [ ]:
mid_activity_opp = ['Other', 'Open Door 1', 'Open Door 2', 'Close Door 1',
                    'Close Door 2', 'Open Fridge', 'Close Fridge', 
                    'Open Dishwasher', 'Close Dishwasher', 'Open Drawer 1', 
                    'Close Drawer 1', 'Open Drawer 2', 'Close Drawer 2', 
                    'Open Drawer 3', 'Close Drawer 3', 'Clean Table', 
                    'Drink from Cup', 'Toggle Switch']